# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# 로드
- 등록정보&제원정보(말소적용)
- 저감장치 부착이력

In [3]:
ods_fold = 'D:/data/big2/'

In [4]:
# about 
# 등록&제원 정보(말소적용)
grade = 5
df = pd.read_csv(os.path.join(ods_fold, f'[ODS][G{grade}]등록정보&제원정보(말소제거)(중복차대제거)(2022.12.29).csv'), low_memory=False)
# 저감장치 부착이력(중복 차대번호 처리)
dpf = pd.read_csv(os.path.join(ods_fold, '[ODS]저감장치부착이력(중복차대제거)(2022.12.29).csv'), low_memory=False, encoding='cp949')

In [5]:
euro = pd.read_excel('D:/과업/[한국환경공단]자동차_환경_빅데이터_구축_2차/data/dpf.xlsx')

# [전처리]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1154696 entries, 0 to 1154695
Data columns (total 49 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   차량관리번호        1154696 non-null  object 
 1   차량등록번호        1154696 non-null  object 
 2   본거지법정동코드      1154696 non-null  int64  
 3   소유자구분         1154696 non-null  object 
 4   차종_등록정보       1154684 non-null  object 
 5   용도            1154696 non-null  object 
 6   현재회원번호        1146027 non-null  object 
 7   차대번호          1154696 non-null  object 
 8   최초등록일자        1154696 non-null  int64  
 9   차량연식          1154696 non-null  int64  
 10  차량제작일자        959021 non-null   float64
 11  배출가스검사유효일자    1154184 non-null  float64
 12  제원관리번호        1154696 non-null  object 
 13  배출가스인증번호      542912 non-null   object 
 14  배출가스등급        1154696 non-null  object 
 15  저공해조치YN(미사용)  1154622 non-null  object 
 16  저공해조치종류       361615 non-null   float64
 17  저공해조치구조변경YN   1154696 non-n

In [7]:
df.shape

(1154696, 49)

In [8]:
dpf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809255 entries, 0 to 809254
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   저감장치종류      585410 non-null  object 
 1   TDMS부착여부    204903 non-null  object 
 2   구조변경일자      809255 non-null  int64  
 3   부착업체명       800321 non-null  object 
 4   부착일자        809254 non-null  float64
 5   탈거승인일       169 non-null     float64
 6   차량번호        809255 non-null  object 
 7   차대번호        809255 non-null  object 
 8   저감장치반납여부    324650 non-null  object 
 9   저감장치구분      809255 non-null  object 
 10  삼원촉매장치반납여부  13688 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 67.9+ MB


In [9]:
dpf.shape

(809255, 11)

In [10]:
euro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   배출가스인증번호  82 non-null     object
 1   DPF유무     82 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB


In [11]:
euro.shape

(82, 2)

## 차대번호 기준 병합

In [12]:
total = df.merge(dpf, on='차대번호', how='left')
total.shape

(1154696, 59)

In [13]:
total['구조변경일자'].isnull().sum()

956990

## 저감 데이터에서 등록&제원 데이터와 매칭되는 차대번호 샘플

In [14]:
miss_match_vin = list(set(dpf['차대번호']) - set(df['차대번호']))
len(miss_match_vin)

611549

In [15]:
match_vin = list(set(dpf['차대번호']) & set(df['차대번호']))
len(match_vin)

197706

In [16]:
match_vin[:5]

['KMCGA17FPTC104099',
 'KNGDNM9N15K210878',
 'KMFGA17FPVC124530',
 'KNCSD0323XS441710',
 '450234313997']

In [17]:
for i in dpf['차대번호'].index:
    print(i)
    break

0


In [18]:
match_dpf_idx = {k:v for k, v in tqdm(zip(dpf['차대번호'], dpf['차대번호'].index))}

809255it [00:00, 1618828.03it/s]


In [19]:
dict(list(match_dpf_idx.items())[:5])

{'*5ZK00529*': 0, '0': 1, '000039': 2, '000128': 3, '000223': 4}

In [20]:
idx = [match_dpf_idx[one] for one in tqdm(match_vin)]

100%|██████████| 197706/197706 [00:00<00:00, 1262649.99it/s]


In [21]:
match_dpf = dpf.loc[idx]
match_dpf.shape

(197706, 11)

In [22]:
match_dpf.sort_values(['차대번호', '구조변경일자', '부착일자'])

,저감장치종류,TDMS부착여부,구조변경일자,부착업체명,부착일자,탈거승인일,차량번호,차대번호,저감장치반납여부,저감장치구분,삼원촉매장치반납여부
89,복합중형,NaN,20090430,아이엠모터스,20090430.00,NaN,서울82루4842,110034227146,NaN,1종,NaN
90,자연대형,NaN,20060203,인천스카니아(인천),20060202.00,NaN,서울83거5222,110034326224,N,1종,NaN
91,자연대형,NaN,20060202,인천스카니아(인천),20060202.00,NaN,서울83거5245,110034326255,N,1종,NaN
92,자연대형,NaN,20060203,인천스카니아(인천),20060202.00,NaN,서울83거5231,110034326256,N,1종,NaN
97,자연대형,NaN,20070412,㈜남양공업사,20070412.00,NaN,경기91바2561,110132228396,N,1종,NaN
...,...,...,...,...,...,...,...,...,...,...,...
809077,자연대형,NaN,20070721,NaN,20070721.00,NaN,경기99바9540,YV2J4CFCO4A580562,N,1종,NaN
809079,자연대형,NaN,20051025,형제자동차공업사,20051025.00,NaN,경기99바3630,YV2J4CFCX3A565422,N,1종,NaN
809080,자연대형,NaN,20070712,NaN,20070712.00,NaN,인천98아4007,YV2J4CFD73A556699,N,1종,NaN
809081,자연대형,NaN,20050923,신동성자동차공업사,20050923.00,NaN,경기94자1417,YV2J4CMA02A551058,N,1종,NaN


In [23]:
match_dpf.shape

(197706, 11)

In [24]:
match_dpf.duplicated('차대번호').sum()

0

### [출력] 매칭되는 차대번호 샘플

In [25]:
# about 35s
match_dpf.sort_values(['차대번호', '구조변경일자', '부착일자']).to_excel(f'analysis2/[BD3][G{grade}][통계]저감장치부착이력_등록&제원정보_매칭_차대번호_샘플(말소제거).xlsx', index=False)

## 분류 통계
- 중복 차대번호 제거
    - 최초등록일자, 배출가스검사유효일자 : 최신순(ascending=False)
- 분류
    - 연료 : 휘발유가스, 경유
    - 차량제작일자
    - 구조변경일자 여부

In [26]:
total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '저감장치종류',
       'TDMS부착여부', '구조변경일자', '부착업체명', '부착일자', '탈거승인일', '차량번호', '저감장치반납여부',
       '저감장치구분', '삼원촉매장치반납여부'],
      dtype='object')

### 중복 차대번호 처리
- '최초등록일자', '배출가스검사유효일자', ascending=False

In [27]:
total.shape[0], len(total['차대번호'].unique())

(1154696, 1154696)

In [28]:
total['dupl'] = total['차대번호'].duplicated().to_list()
total.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,현재회원번호,차대번호,최초등록일자,차량연식,...,TDMS부착여부,구조변경일자,부착업체명,부착일자,탈거승인일,차량번호,저감장치반납여부,저감장치구분,삼원촉매장치반납여부,dupl
0,KL3C3AHF27K000082-01,92두9517,4481025000,법인,화물,자가용,CAE5CB3DD606A73E07AC169E171B8052,KL3C3AHF27K000082,20220711,2007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,KL3C3AHF27K000068-01,84라1558,4372033032,주민,화물,자가용,E3D34867F18FF514DC235FE97CF8B726,KL3C3AHF27K000068,20220413,2007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,1P3EW65G5XV503775-01,50더0491,4157010600,법인,승용,자가용,B9638F38C154884829E69103F64C7EE4,1P3EW65G5XV503775,20211124,1999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,1P3EW65G5YV606177-01,67거4988,5011013700,주민,승용,자가용,3BF0F5EEEB8EB9C1D08D567DC65098ED,1P3EW65G5YV606177,20211124,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,KMCYSS7JP7U084697-01,99도4857,4423038028,주민,특수,자가용,955AC8A2055B73FE92273C8624CE9FDA,KMCYSS7JP7U084697,20210826,2007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [29]:
dupl_vin = total[total['dupl'] == True].sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=False).reset_index(drop=True)
dupl_vin.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,현재회원번호,차대번호,최초등록일자,차량연식,...,TDMS부착여부,구조변경일자,부착업체명,부착일자,탈거승인일,차량번호,저감장치반납여부,저감장치구분,삼원촉매장치반납여부,dupl


#### [출력] 중복 차대번호 샘플

In [30]:
# dupl_vin.to_excel(f'analysis2/[BD3][G{grade}][샘플]등록&제원-저감_중복차대번호(말소제거).xlsx')

In [31]:
total1 = total.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
total1.shape

(1154696, 60)

## 유로4 DPF 부착여부 적용

In [32]:
total2 = total1.merge(euro, on='배출가스인증번호', how='left')
total2.shape

(1154696, 61)

In [33]:
total2['DPF유무'].value_counts()

무    36
Name: DPF유무, dtype: int64

In [34]:
total2['DPF유무'] = total2['DPF유무'].astype('str')
total2['DPF유무'].value_counts()

nan    1154660
무           36
Name: DPF유무, dtype: int64

In [35]:
total2['DPF유무'] = total2['DPF유무'].str.replace('nan', '무')
total2['DPF유무'].value_counts()

무    1154696
Name: DPF유무, dtype: int64

# [분석]

## [Case 5]

### 차량연료별 분류
- 차량 대수
    - 휘발류•가스('휘발유', '알코올', 'LGP(액화석유가스)', 'CNG(압축천연가스)')
    - 경유

In [36]:
total2['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', 'LGP(액화석유가스)', nan, '기타연료', '알코올'],
      dtype=object)

In [37]:
total2[ (total2['연료'] == '휘발유') | (total2['연료'] == '알코올') | (total2['연료'] == 'LGP(액화석유가스)') | (total2['연료'] == 'CNG(압축천연가스)') ].shape

(30482, 61)

In [38]:
total2[total2['연료'] == '경유'].shape

(1124162, 61)

In [39]:
num_by_fuel = pd.DataFrame({'연료':['휘발유_가스', '경유'], 
                            '수(차대번호)':[
                                total2[ (total2['연료'] == '휘발유') | (total2['연료'] == '알코올') | (total2['연료'] == 'LGP(액화석유가스)') | (total2['연료'] == 'CNG(압축천연가스)') ].shape[0], 
                                total2[total2['연료'] == '경유'].shape[0]
                                ]
                            })
num_by_fuel

,연료,수(차대번호)
0,휘발유_가스,30482
1,경유,1124162


In [40]:
fuel_list = []
for one in tqdm(total2['연료']):
    if one == '휘발유' or one == '알코올' or one == 'LGP(액화석유가스)' or one == 'CNG(압축천연가스)':
        fuel_list.append('휘발유_가스')
    elif one == '경유':
        fuel_list.append('경유')
    else:
        fuel_list.append('기타')
len(fuel_list)

100%|██████████| 1154696/1154696 [00:00<00:00, 1622728.25it/s]


1154696

In [41]:
total2['fuel'] = fuel_list

### 차량제작일자별 분류
- 휘발류•가스('휘발유', 'LGP(액화석유가스)', '알코올', 'CNG(압축천연가스)')
    - A : 1988.01.01 ~ 2000.12.31
    - B : ~ 1997.12.31
- 경유(DPF : 구조변경일자 여부)
    - A : DPF, 2008.01.01 ~
    - B : DPF, ~ 2007.12.31
    - C : 2008.01.01 ~
    - D : ~ 2007.12.31

In [42]:
total2['차량제작일자'].describe()

count     959021.00
mean    20030337.35
std        42351.43
min      2210413.00
25%     20011230.00
50%     20040411.00
75%     20060119.00
max     20200106.00
Name: 차량제작일자, dtype: float64

In [43]:
total2['구조변경일자'].describe()

count     197706.00
mean    20137920.69
std        50938.75
min     20041010.00
25%     20090512.00
50%     20140530.00
75%     20190814.00
max     20200305.00
Name: 구조변경일자, dtype: float64

In [44]:
total2['구조변경일자'].isnull().sum()

956990

In [45]:
total2['저감장치종류'].unique()

array([nan, '복합소형', '복합중형', '소형', 'DOC', '복합대형', '중형', 'pDPF', '자연대형+SCR',
       '자연대형', '자연중형'], dtype=object)

In [46]:
total2['저감장치종류'].isnull().sum()

976776

In [47]:
str(np.nan) == 'nan'

True

In [48]:
grade_list = []
for f, y, d, e in tqdm(zip(total2['연료'], total2['차량제작일자'], total2['구조변경일자'], total2['DPF유무'])):
    if ( (f == '휘발유') or (f == 'LGP(액화석유가스)') or (f == '알코올') or (f == 'CNG(압축천연가스)') ) and (19980101 <= y <= 20001231):
        grade_list.append('A')
    elif (f == '휘발유' or f == 'LGP(액화석유가스)' or f == '알코올' or f == 'CNG(압축천연가스)') and (y <= 19971231):
        grade_list.append('B')
    elif (f == '경유') and (y >= 20080101) and ( (str(d) != 'nan') or e == '유'):
        grade_list.append('A')
    elif (f == '경유') and (y <= 20071231) and ( (str(d) != 'nan') or e == '유'):
        grade_list.append('B')
    elif (f == '경유') and (y >= 20080101) and ( (str(d) == 'nan') or e == '무'):
        grade_list.append('C')
    elif (f == '경유') and (y <= 20071231) and ( (str(d) == 'nan') or e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

1154696it [00:02, 496357.33it/s]


1154696

In [49]:
total2['Grade'] = grade_list

In [50]:
total2.groupby(['fuel'])['차대번호'].count()

fuel
경유        1124162
기타             52
휘발유_가스      30482
Name: 차대번호, dtype: int64

In [51]:
total2.groupby(['fuel', 'Grade'])['차대번호'].count()

fuel    Grade
경유      A            35
        B        195033
        C           227
        D        762799
        X        166068
기타      X            52
휘발유_가스  A            24
        B           871
        X         29587
Name: 차대번호, dtype: int64

In [52]:
total2[(total2['fuel'] == '경유') & (total2['Grade'] == 'X')]

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,현재회원번호,차대번호,최초등록일자,차량연식,...,부착일자,탈거승인일,차량번호,저감장치반납여부,저감장치구분,삼원촉매장치반납여부,dupl,DPF유무,fuel,Grade
266612,KN98K6MGD5Z1BE009-01,80노8908,1150010300,사업자,화물,관용,8C4BF90DEA4EA30879FEA3A8DE3FC71D,KN98K6MGD5Z1BE009,20051223,2005,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,경유,X
506642,KP95J6MJD4CB01203-01,98가9553,1154510300,사업자,특수,자가용,0CA51720B23140C44101322F40E0C783,KP95J6MJD4CB01203,20040220,2004,...,20110811.00,NaN,98가9553,NaN,1종,NaN,False,무,경유,X
669166,4199경기90바2561,85서7047,4148032000,법인,화물,자가용,400C8EA721A0EA012972C26B4A0FF3C5,KMFGA17LP2C705790,20020729,2002,...,20080929.00,NaN,경기90바2561,NaN,2종,NaN,False,무,경유,X
686085,KNCSD0112VS362476-01,경남80무2604,3171025032,주민,화물,자가용,5880360878D5FA1F571AD1E7430074FE,KNCSD0112VS362476,20020524,1997,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,경유,X
686554,4199경기80바2280,87버2408,4141010500,주민,화물,자가용,A9380923948ACAAFA07F291507F98AE5,KMFLA19AP2C031586,20020522,2002,...,20060804.00,NaN,경기80바2280,N,1종,NaN,False,무,경유,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154687,000000KW45C001134-01,85버2435,4277025023,주민,화물,자가용,882D4E1D21042D3834DDE09D0FE39603,000000KW45C001134,19620629,1945,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,경유,X
1154688,KW55S000593-01,90모8325,4182034521,주민,화물,자가용,5F1CE533E1194B052AC7C34C1A448775,KN90K7MKDFKZ00001,19620629,1955,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,경유,X
1154690,kw44c000772-01,강원82자8830,4215031026,주민,화물,영업용,E2E2D87F480CE18599C1AD7EE999BA63,KW44C000772,19620625,1962,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,경유,X
1154692,04441201291-01,전북7가6256,4573025021,주민,화물,자가용,2DCFEA9B4CF27C189548395F4BC897EA,04441201291,19620306,1944,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,경유,X


In [53]:
total2[(total2['fuel'] == '휘발유_가스') & (total2['Grade'] == 'X')].head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,현재회원번호,차대번호,최초등록일자,차량연식,...,부착일자,탈거승인일,차량번호,저감장치반납여부,저감장치구분,삼원촉매장치반납여부,dupl,DPF유무,fuel,Grade
2,1P3EW65G5XV503775-01,50더0491,4157010600,법인,승용,자가용,B9638F38C154884829E69103F64C7EE4,1P3EW65G5XV503775,20211124,1999,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,휘발유_가스,X
3,1P3EW65G5YV606177-01,67거4988,5011013700,주민,승용,자가용,3BF0F5EEEB8EB9C1D08D567DC65098ED,1P3EW65G5YV606177,20211124,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,휘발유_가스,X
297,KL90C3AHGGZ2CK001-01,20노6890,1168010500,주민,승용,자가용,B733C838FBA5D9A1D2DD1375897F635B,KL90C3AHGGZ2CK001,20131105,1986,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,휘발유_가스,X
468,WP0ZZZ91ZFS102316-01,58마9300,1117013000,주민,승용,자가용,76D5B230A2F7CFC4AC35B2B388BE554B,WP0ZZZ91ZFS102316,20100616,1985,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,휘발유_가스,X
477,WDBBA48D0GA039761-01,15오2546,1144012300,주민,승용,자가용,BEF4E10BA4DC22E0F086C9E6B947DE6D,WDBBA48D0GA039761,20100204,1986,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,휘발유_가스,X


In [54]:
total2[(total2['fuel'] == '기타') & (total2['Grade'] == 'X')].head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,현재회원번호,차대번호,최초등록일자,차량연식,...,부착일자,탈거승인일,차량번호,저감장치반납여부,저감장치구분,삼원촉매장치반납여부,dupl,DPF유무,fuel,Grade
833421,449934394867-02,전남91바9076,4684025600,법인,화물,영업용,8D0EF2C7D6EABE7761ECE5AB8E2C323B,449934394867,19991023,1999,...,20060830.00,NaN,89어7365,N,1종,NaN,False,무,기타,X
837531,419934399342-01,부산91사5271,2629011000,사업자,화물,영업용,0B2997A25307E6F5E1DD8FDB6D25430C,419934399342,19990914,1999,...,20080630.00,NaN,경기91바6972,NaN,1종,NaN,False,무,기타,X
840053,419934399280-02,경북81아1763,4719025600,법인,화물,영업용,170D038E0DD85EACD12E1AB4BB2072D6,419934399280,19990820,1999,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,기타,X
1051132,KN3LAP5C1MK101195-01,경남7고6070,4812514000,주민,화물,자가용,48631EA8EAD02F04737E81B09078671D,KN3LAP5C1MK101195,19910228,1991,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,기타,X
1068213,ZFA15400000258139-01,서울52고9225,1168011800,법인,승용,자가용,5D6D4DD77C4AE6E3A7FD7D0663447043,ZFA15400000258139,19900518,1990,...,NaN,NaN,NaN,NaN,NaN,NaN,False,무,기타,X


#### [출력] 통계

In [55]:
with pd.ExcelWriter(f'analysis2/[BD3][G{grade}][통계]Case5_등급분류(중복차대제거)(말소제거).xlsx') as writer:
    total2.groupby(['fuel'])['차대번호'].count().to_excel(writer, sheet_name='연료별')
    total2.groupby(['fuel', 'Grade'])['차대번호'].count().to_excel(writer, sheet_name='등급별')

#### [출력] 등급 산정 안된 샘플

In [56]:
# about 3m
with pd.ExcelWriter(f'analysis2/[BD3][G{grade}][샘플]Case5_등급미분류(중복차대제거)(말소제거).xlsx') as writer2:
    total2[(total2['fuel'] == '경유') & (total2['Grade'] == 'X')].to_excel(writer2, sheet_name='경유', index=False)
    total2[(total2['fuel'] == '휘발유_가스') & (total2['Grade'] == 'X')].to_excel(writer2, sheet_name='휘발유', index=False)
    total2[(total2['fuel'] == '기타') & (total2['Grade'] == 'X')].to_excel(writer2, sheet_name='기타', index=False)

# 코드 마지막